In [1]:
# import pandas_profiling as ppf
# dataset.info()
# report = ppf.ProfileReport(dataset[['Adj Close','Volume']])
# report.to_file('report_AdjCloseVolume.md')

In [2]:
import pandas as pd
import torch
import numpy as np
import datetime
import matplotlib.pyplot as plt
from matplotlib.pylab import rcParams
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler ##数据归一化

output_size = 7

In [3]:
def preprocess(data,m,n):
    '''
    data: the dataframe of stock price
    m: 前m天预测
    n: 预测几天
    '''
    minMax = MinMaxScaler()    
    data_transformed = minMax.fit_transform(data)

    adj_close = data["Adj Close"].tolist()
    adj_volume = data_transformed[:,-2:]
    #
    res_X = []
    res_y = []
    
    for i in range(0,len(adj_close)-m-n+1):
        res_X.append(adj_volume[i:i+m])
        res_y.append(adj_close[i+m:i+m+n])
    return res_X,res_y,adj_volume[-m:]

In [4]:
dataset = pd.read_csv('/Users/elvis/ITProjects/GitHub/PythonTask/100dayML/100_day_ml_py/ml_project/raw_price_train/1_r_price_train.csv',index_col='Date')
dataset.index = pd.to_datetime(dataset.index)

In [5]:
X, y, final_pred = preprocess(dataset,14,output_size)
X_train,X_test,y_train,y_test = train_test_split(X,y,train_size=0.7)
# print((np.array(X_train)).shape)
# print((np.array(y_train)).shape)
# print((np.array(X)).shape)
# print((np.array(y)).shape)
# print((np.array(y[-1])).shape)
print(np.array(final_pred).shape)

(14, 2)


In [6]:
import torch.utils.data as Data

tensor_x_train = torch.Tensor(X_train)
tensor_y_train = torch.Tensor(y_train)

tensor_x_test = torch.Tensor(X_test)
tensor_y_test = torch.Tensor(y_test)

# 先转换成 torch 能识别的 Dataset
torch_train_dataset = Data.TensorDataset(tensor_x_train, tensor_y_train)
torch_test_dataset = Data.TensorDataset(tensor_x_test, tensor_y_test)

# 把 dataset 放入 DataLoader
train_loader = Data.DataLoader(
    dataset=torch_train_dataset,      # torch TensorDataset format
    batch_size=100,      # mini batch size
    shuffle=True,               # 要不要打乱数据 (打乱比较好)
    num_workers=2,              # 多线程来读数据
)
test_loader = Data.DataLoader(
    dataset=torch_test_dataset,      # torch TensorDataset format
    batch_size=100,      # mini batch size
    shuffle=True,               # 要不要打乱数据 (打乱比较好)
    num_workers=2,              # 多线程来读数据
)

In [7]:
for step, (batch_x, batch_y) in enumerate(train_loader):
    print(batch_x.size())

torch.Size([100, 14, 2])
torch.Size([100, 14, 2])
torch.Size([100, 14, 2])
torch.Size([100, 14, 2])
torch.Size([100, 14, 2])
torch.Size([66, 14, 2])


In [8]:
import torch.nn as nn
class RNN(nn.Module):
    def __init__(self):
        super(RNN, self).__init__()

        self.rnn = nn.LSTM(  # if use nn.RNN(), it hardly learns
            input_size=2,
            hidden_size=64,         # rnn hidden unit
            num_layers=1,           # number of rnn layer
            batch_first=True,       # input & output will has batch size as 1s dimension. e.g. (batch, time_step, input_size)
        )
        

        self.out = nn.Linear(64, output_size)

    def forward(self, x):
        # x shape (batch, time_step, input_size)
        # r_out shape (batch, time_step, output_size)
        # h_n shape (n_layers, batch, hidden_size)
        # h_c shape (n_layers, batch, hidden_size)
        r_out, (h_n, h_c) = self.rnn(x, None)   # None represents zero initial hidden state

        # choose r_out at the last time step
#         print('r_out[:, -1, :]',r_out[:, -1, :].size())
        out = self.out(r_out[:, -1, :])
        return out
    
class BILSTM(nn.Module):
    def __init__(self):
        super().__init__()
        
        self.bilstm = nn.LSTM(
            input_size=2,
            hidden_size=64,         # rnn hidden unit
            num_layers=2,
            bidirectional=True,
            batch_first=True,
            dropout=0.2)
        self.fc = nn.Sequential(
#             nn.Linear(64*2, 32),
#             nn.Dropout(0.2),
#             nn.ReLU(),
            nn.Linear(64*2,output_size),
        )

    def forward(self, x):
        r_out, (h_n, h_c) = self.bilstm(x, None)   # None represents zero initial hidden state
#         print('r_out',r_out.size())
#         print('h_n',h_n.size())
#         print('h_n[-2, :, :]',h_n[-2, :, :].size())
#         print('h_n[-2, :, :],h_n[-1, :, :]),dim=1:',torch.cat((h_n[-2, :, :],h_n[-1, :, :]),dim=1).size())
        output = self.fc(torch.cat((h_n[-2, :, :],h_n[-1, :, :]),dim=1))
        return output

In [9]:
model = BILSTM()
import torch.optim as optim

# 定义优化器
optimizer = optim.Adam(model.parameters())
loss_func = nn.MSELoss()

In [10]:
from sklearn.metrics import mean_squared_error
def train(model, loader, optimizer, loss_func):
    model.train()

    for step, (batch_x, batch_y) in enumerate(loader):  # 每一步 loader 释放一小批数据用来学习
        # 训练的地方...
        batch_x = batch_x.view(-1, 14, 2)              # reshape x to (batch, time_step, input_size)
        batch_y = batch_y.view(-1,output_size)
        output = model(batch_x)
        loss = loss_func(output, batch_y)                   # cross entropy loss
        
        optimizer.zero_grad()                           # clear gradients for this training step
        loss.backward()                                 # backpropagation, compute gradients
        optimizer.step()
        
        acc = mean_squared_error(batch_y.numpy().tolist(),output.detach().numpy())
        
    return loss.data.numpy(), acc

In [11]:
# 不用优化器了
def evaluate(model, loader, loss_func):
    # 转成测试模式，冻结dropout层或其他层
    model.eval()

    with torch.no_grad():
        for step, (batch_x, batch_y) in enumerate(loader):  # 每一步 loader 释放一小批数据用来学习

            batch_x = batch_x.view(-1, 14, 2)              # reshape x to (batch, time_step, input_size)
            batch_y = batch_y.view(-1,output_size)
            output = model(batch_x)
            loss = loss_func(output, batch_y)
            acc = mean_squared_error(batch_y.numpy().tolist(),output.detach().numpy())

        # 调回训练模式
        model.train()

    return acc

In [31]:
best_test_mse = float('inf')
for epoch in range(10):   # 训练所有!整套!数据 10 次
    train_loss, train_mse = train(model, train_loader, optimizer, loss_func)
    test_mse = evaluate(model, test_loader, loss_func)
    
    
    if test_mse < best_test_mse:
        best_test_mse = test_mse
        torch.save(model.state_dict(), 'stock-bilstm-model.pt')
        
    if epoch> 0 :
        print('Epoch: ', epoch, '| train_loss: ', train_loss, '| train_mse x: ', train_mse, '| test_mse x: ', test_mse)
        # 打出来一些数据
#         print('Epoch: ', epoch, '| Step: ', step, '| batch x: ',
#               batch_x.numpy(), '| batch y: ', batch_y.numpy())

Epoch:  1 | train_loss:  1.0793341 | train_mse x:  1.0793340979147126 | test_mse x:  5.08605900586121
Epoch:  2 | train_loss:  0.86512715 | train_mse x:  0.8651271472026172 | test_mse x:  3.7112915856301822
Epoch:  3 | train_loss:  0.987288 | train_mse x:  0.9872879806732558 | test_mse x:  1.7705096206845827
Epoch:  4 | train_loss:  1.1776162 | train_mse x:  1.177616202949548 | test_mse x:  4.9356660207053205
Epoch:  5 | train_loss:  1.0497309 | train_mse x:  1.049730915724605 | test_mse x:  4.40104284277355
Epoch:  6 | train_loss:  1.0617707 | train_mse x:  1.061770731538952 | test_mse x:  3.488451821844264
Epoch:  7 | train_loss:  1.0516388 | train_mse x:  1.0516388668032117 | test_mse x:  2.2061509402963178
Epoch:  8 | train_loss:  0.8874944 | train_mse x:  0.8874943569010325 | test_mse x:  2.770129374686441
Epoch:  9 | train_loss:  0.89888406 | train_mse x:  0.8988840632060696 | test_mse x:  4.445307321153892


In [32]:
model.load_state_dict(torch.load('stock-bilstm-model.pt'))
pred = model(torch.Tensor(final_pred).view(1,14,2))
print(pred)

tensor([[106.6203, 106.3472, 105.8384, 106.2928, 107.1255, 107.6579, 107.7180]],
       grad_fn=<AddmmBackward>)
